# Is your house worth it's weight in gold?

Now we've calculated how much a given house rose in value over time - the question is, would you have been better off buying gold?

In the `data` folder, we have the daily price of one ounce of gold in USD, called `XAUUSD` in trading terms (e`X`change rate of `AU` aka Gold to `USD`)

We want to find out what would have happened if instead of buying a house, we would have bought an equivalent sum of gold. Finally we want to compare what the average profit would be if everyone had invested in gold instead of buying a house.

For simplicity, use the daily Closing price.

```{note}
Since gold is priced in USD, and our houseprices are in GBP, there is also a USD/GBP daily prices in the `data/fx` folder to convert between currencies.
```

Breaking it down:

1. Load daily gold prices into Iceberg - for bonus points, apply a yearly partitioning scheme
2. Load exchange rates into Iceberg - for bonus points, apply a yearly partitioning scheme
3. Convert daily gold prices from USD to GBP
4. For each row in our `profits` table, calculate the equivalent amount of gold they would have been able to purchase on that date
5. Calculate the total value of that amount of gold on the sell date
6. Calculate the profit of our gold trade
7. Compare to Gold profit with Housing profit

In [30]:
import polars as pl
from utils import catalog, engine
from pyiceberg.schema import Schema, NestedField
from pyiceberg.types import DecimalType, DateType, StringType
from pyiceberg.partitioning import PartitionSpec, YearTransform, PartitionField
from IPython.display import display
pl.Config.set_thousands_separator(",")

polars.config.Config

## 1. Load daily gold prices into Iceberg
Start by creating the table in Iceberg. To organize things a bit better, I'm creating a new namespace `commodities` - we could imagine putting a table for oil prices or silver in here

In [2]:
catalog.create_namespace_if_not_exists("commodities")

gold_schema = Schema(
    NestedField(1, "date", DateType(), required=True, doc="Day of recorded price"),
    NestedField(
        2,
        "price",
        DecimalType(precision=38, scale=2),
        required=True,
        doc="Price in USD of one ounce of gold",
    ),
    identifier_field_ids=[1],
)

gold_prices_t = catalog.create_table_if_not_exists(
    "commodities.gold",
    schema=gold_schema,
    partition_spec=PartitionSpec(
        PartitionField(
            source_id=1, field_id=1, transform=YearTransform(), name="date_year"
        )
    ),
)

Next, read in the CSV, picking out the two columns we care about, remembering to convert to the correct schema, 

In [3]:
gold_prices = (
    pl.scan_csv("data/gold/daily_gold_prices.csv", separator=";", try_parse_dates=True)
    .select(pl.col("Date").alias("date"), pl.col("Close").alias("price"))
    .collect()
)

In [27]:
gold_prices

date,price
datetime[μs],f64
2004-06-11 00:00:00,384.1
2004-06-14 00:00:00,382.8
2004-06-15 00:00:00,388.6
2004-06-16 00:00:00,383.8
2004-06-17 00:00:00,387.6
…,…
2025-01-28 00:00:00,"2,763.17"
2025-01-29 00:00:00,"2,759.68"
2025-01-30 00:00:00,"2,794.06"


In [5]:
gold_prices_t.append(gold_prices.to_arrow().cast(gold_schema.as_arrow()))

# 2. Load daily exchange rates into Iceberg

Similar process - create a new namespace and load the fx rates. Since this is technically a table of currency pairs, we can make the schema a bit more future-proof

In [6]:
catalog.create_namespace_if_not_exists("fx")

fx_schema = Schema(
    NestedField(1, "date", DateType(), required=True, doc="Day of recorded price"),
    NestedField(
        2,
        "from",
        StringType(),
        required=True,
        doc="From currency code of the currency pair",
    ),
    NestedField(
        3,
        "to",
        StringType(),
        required=True,
        doc="To currency code of the currency pair",
    ),
    NestedField(
        4,
        "exchange_rate",
        DecimalType(precision=38, scale=4),
        required=True,
        doc="Exchange rate of the currency pair",
    ),
    identifier_field_ids=[1, 2, 3],
)

Next, reading in the CSV file and transforming slightly to conform to our schema

In [7]:
fx_table = catalog.create_table_if_not_exists("fx.rates", schema=fx_schema)

In [9]:
fx_rates = (
    pl.scan_csv("data/fx/USD_GBP.csv", try_parse_dates=True)
    .with_columns(pl.lit("USD").alias("from"), pl.lit("GBP").alias("to"))
    .select(
        pl.col("date"),
        pl.col("from"),
        pl.col("to"),
        pl.col("close").alias("exchange_rate"),
    )
    .collect()
)
fx_rates

date,from,to,exchange_rate
date,str,str,f64
2025-05-16,"""USD""","""GBP""",0.7529
2025-05-15,"""USD""","""GBP""",0.7516
2025-05-14,"""USD""","""GBP""",0.754
2025-05-13,"""USD""","""GBP""",0.7516
2025-05-12,"""USD""","""GBP""",0.759
…,…,…,…
2006-03-07,"""USD""","""GBP""",0.576
2006-03-06,"""USD""","""GBP""",0.5713
2006-03-03,"""USD""","""GBP""",0.5696


In [10]:
fx_table.append(fx_rates.to_arrow().cast(fx_schema.as_arrow()))

Next comes the business logic - the actual work of figuring out how much gold we could have bought and how much that would have sold for

In [23]:
sql = """
-- Convert USD gold prices into GBP denominated prices
with gold_prices as (
    select commodities.gold.date as gold_date, commodities.gold.price * fx.rates.exchange_rate as gold_price
    from commodities.gold
    join fx.rates on fx.rates.date = commodities.gold.date
), gold_purchase as (
-- Calculate how much gold (in ounces) the house owner could have bought at the time of purchase
    select address_id, 
    first_price / gold_price as purchased_gold
    from housing.profits
    join gold_prices on gold_date = housing.profits.first_day
), gold_sell as (
-- Calculate how much that amount of gold is worth at the time of sale, as well as the profit
    select housing.profits.address_id,
    cast((purchased_gold * gold_price) - first_price as DECIMAL(38, 2)) as gold_profit,
    cast(profit as DECIMAL(38, 2)) as house_profit
    from housing.profits
    join gold_purchase on gold_purchase.address_id = housing.profits.address_id
    join gold_prices on gold_prices.gold_date = last_day
)
select * from gold_sell
"""

In [24]:
gold_vs_house_profits = pl.read_database(sql, engine)
gold_vs_house_profits

address_id,gold_profit,house_profit
str,"decimal[38,2]","decimal[38,2]"
"""E7212FC4DD4DE6EEACEF3A2E069ACC…",40230.82,-22500.00
"""C93F0B6F6F82401AFF1C511FF7ACEF…",15498.03,39500.00
"""CE92C6E47E7B015B3A80A50C31AC79…",111327.95,85000.00
"""97F1F1EB870F31583D8E6D74053B4C…",39107.51,100500.00
"""996AD2EF5EFB730D42202B131BEB84…",77923.33,200000.00
…,…,…
"""A74FFC2E0A9FEEE17E5C4657D3B1E0…",251123.75,125005.00
"""315834E5DF9EEFA910006FADC8FFBD…",98100.73,230000.00
"""4470A4BEBFF547332D4749230B61DB…",108148.39,25000.00


Now we have a per-address calculation, let's summarize the results

In [28]:
gold_vs_house_profits.select(pl.all().exclude("address_id")).describe()

statistic,gold_profit,house_profit
str,f64,f64
"""count""",1.156815e6,1.156815e6
"""null_count""",0.0,0.0
"""mean""","131,763.818885","62,487.478001"
"""std""","174,111.621751","235,601.719894"
"""min""","-917,359.06",-1.198e7
"""25%""","44,106.93","20,000.0"
"""50%""","90,281.03","44,500.0"
"""75%""","166,059.71","80,000.0"
"""max""","21,050,917.48",2.23847e8


How many percent would have done better buying gold than a house?

In [31]:
with pl.Config(set_tbl_rows=100):
    summary_df = (
        gold_vs_house_profits.select(
            pl.col("house_profit")
            .sub(pl.col("gold_profit"))
            .qcut(100, labels=[f"Q{i + 1}" for i in range(100)], include_breaks=True)
        )
        .unnest("house_profit")
        .unique()
        .sort("breakpoint")
    )
    display(summary_df)

breakpoint,category
f64,cat
"-590,355.966","""Q1"""
"-445,956.9788","""Q2"""
"-374,034.1702","""Q3"""
"-327,544.9948","""Q4"""
"-293,984.913","""Q5"""
"-268,339.6008","""Q6"""
"-247,603.4614","""Q7"""
"-230,286.2124","""Q8"""
"-215,625.6432","""Q9"""


# Exercise: What does it look like in your region?

This is the picture for all of the UK - what does it look like for your region?